In [1]:
import os
import pandas as pd
import numpy as np
import csv
import sys
import codecs
import pyodbc
import urllib
import sqlalchemy
pd.options.display.max_columns = None
from tqdm import tqdm

In [2]:
input_path = r""

1. Rename files from XLS to CSV

In [18]:
for filename in os.listdir(input_path):
    infilename = os.path.join(input_path,filename)
    if not os.path.isfile(infilename): continue
    oldbase = os.path.splitext(filename)
    newname = infilename.replace('.txt', '.csv')
    output = os.rename(infilename, newname)

2. Create list of file path

In [3]:
#get a list of files and filepaths from our destination folder

#empty list for filepaths
paths = [] 

#empty list for filenames
fns = []

for root, dirs, files in os.walk(input_path):
    for name in files:
        paths.append(os.path.join(root, name))
        fns.append(name)

SET DESTINATION TABLE AND LOADED BY

In [28]:
table_name = 'provide_table_name'
loaded_by = 'provide_name'

3. Create connection strings for SQL table

In [6]:
server = '###.##.##.##' 
database = '#########' 
username = '############' 
password = '#########'

#select appropritate Driver for SQL Connections
driver_name = ''
driver_names = [x for x in pyodbc.drivers() if x.endswith(' for SQL Server')]
print(driver_names)
if driver_names:
    driver_name = driver_names[0]
else:
    print('(No suitable driver found. Cannot connect.)')

params = urllib.parse.quote_plus('DRIVER={}'.format(driver_name)+';SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)


def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

#progressbar
def insert_with_progress(df):
    chunksize = int(len(df) / 20) # 10%
    with tqdm(total=len(df)) as pbar:
        for i, cdf in enumerate(chunker(df, chunksize)):
            replace = "replace" if i == 0 else "append"
            cdf.to_sql(con=engine, name=table_name, if_exists=replace, index=False,schema="dbo")
            pbar.update(chunksize)

#Set cursor.fast_executemany = True using events

@sqlalchemy.event.listens_for(engine, "before_cursor_execute")

def receive_before_cursor_execute(conn, cursor, statement, params, context, executemany):
            if executemany:
                cursor.fast_executemany = True

#Test Connection
# sql = ("select * from testing")
# data = pd.read_sql(sql,engine)
# data

['ODBC Driver 17 for SQL Server', 'ODBC Driver 13 for SQL Server']


4. Provide New Column Names

In [7]:
new_column_names = []

5. Function to Rename Columns

In [8]:
#Funtion to rename columns
def rename_column_name(df,new_column_list):
    old_column_list = new_df.columns.tolist()
    df = df.rename(columns=dict(zip(old_column_list, new_column_list)))
    return df

6. Function to Change Data type of certain columns

In [9]:
#Funtion to change data type of date field and number field
def change_datatype(df):
    cols = df.dtypes.apply(lambda x: x.name).to_dict()

    for c,d in cols.items():
        if "DATE" in c:
            try:
                df[c] = pd.to_datetime(df[c]) #change date field to datetime
            except:
                pass
        elif any(item in c for item in ['AMOUNT','AMT']):
            try:
                mask = df[c].str.endswith('-')
                df.loc[mask, c] = '-' + df.loc[mask, c].str[:-1]
                df[c] = df[c].str.replace(' ','')
                df[c] = df[c].str.replace(',','').apply(pd.to_numeric) #change amount fields to numeric data type
            except:
                raise Exception
        else:
            try:
                df[c] = df[c].convert_dtypes() #change rest of the fields to string
            except:
                raise
    return df

7. Function to drop unwanted columns

In [10]:
def drop_unwanted_col(df):
    
    column_name = df.columns.tolist()
    for c in column_name:
        if  'Unnamed' in c:
            df.drop([c], axis=1, inplace=True)
        #elif any(item in c for item in ['Unnamed:','CoCd']):
            
    return df

7. Read through each file and concat data

In [67]:
appended_data = []

for one_file in paths:

    doc = codecs.open(one_file,'rU','UTF-8')
    
    print(one_file)
    df = pd.read_csv(doc,sep='|',skipinitialspace=True,skiprows=5,low_memory=False,quoting=csv.QUOTE_NONE)
    
    
    #drop extra columns in import
    df = drop_unwanted_col(df)
    
    new_df = df.loc[df['DocumentNo'].notnull()] #drop null in DocumentNo
    new_df = new_df[new_df['DocumentNo'].str.contains('[0-9]')] #filter dataframe where DocumentNo is numeric only2
      
    #Rename Columns
    new_df = rename_column_name(new_df,new_column_names)

    #change datatype
    new_df = change_datatype(new_df)

    #Add filename as column
    new_df['ADV_FILENAME'] = os.path.basename(one_file)
    new_df['ADV_LOADED_BY'] = loaded_by
    new_df['ADV_LOAD_DATE'] = pd.Timestamp.now()

    # store DataFrame in list
    appended_data.append(new_df)
    
    del(new_df)
    del(df)

appended_data = pd.concat(appended_data)

C:\Users\sdash029\Desktop\Project Folder\Project Cardinal\02242021\FY18 1496 OFFSET\2500_2018.csv
C:\Users\sdash029\Desktop\Project Folder\Project Cardinal\02242021\FY18 1496 OFFSET\2504_2018.csv
C:\Users\sdash029\Desktop\Project Folder\Project Cardinal\02242021\FY18 1496 OFFSET\6007_2018.csv
C:\Users\sdash029\Desktop\Project Folder\Project Cardinal\02242021\FY18 1496 OFFSET\6008_2018.csv
C:\Users\sdash029\Desktop\Project Folder\Project Cardinal\02242021\FY18 1496 OFFSET\6013_2018.csv


ParserError: Error tokenizing data. C error: Expected 44 fields in line 58, saw 46


In [29]:
insert_with_progress(appended_data)

1552068it [35:21, 731.70it/s]                                                                                          


8. Testing

In [21]:
doc = codecs.open(one_file,'rU','UTF-8')

df = pd.read_csv(doc,sep='\t',skipinitialspace=True,skiprows=5,low_memory=False,quoting=csv.QUOTE_NONE)

df = drop_unwanted_col(df)

new_df = df.loc[df['DocumentNo'].notnull()] #drop null in DocumentNo columns and deselect 3 columns
new_df = new_df[new_df['DocumentNo'].str.contains('[0-9]')] #filter dataframe where DocumentNo is numeric only2

old_column_names = new_df.columns.tolist()
#Rename Columns
new_df = rename_column_name(new_df, new_column_names)

#change datatype
new_df = change_datatype(new_df)

#Add filename as column
new_df['ADV_FILENAME'] = os.path.basename(one_file)



In [27]:
appended_data.groupby(['ADV_FILENAME'])['ADV_FILENAME'].count()

ADV_FILENAME
2500_2018.csv      1287
2504_2018.csv     72278
6007_2018.csv    457993
6008_2018.csv    411388
6013_2018.csv     11288
6014_2018.csv    397448
6501_2018.csv     75935
6505_2018.csv     44731
6509_2018.csv      3782
8003_2018.csv      2048
Name: ADV_FILENAME, dtype: int64

In [46]:
new_df.to_sql('dash_test',engine,index=False,if_exists="append",schema="dbo")

In [36]:
appended_data[~appended_data['PROFIT_CENTER'].str.contains('P')]
#to_sql('dash_test_1',engine,index=False,if_exists="append",schema="dbo")

,COMPANY_CODE,CUSTOMER,DOCUMENT_NUMBER,DOCUMENT_TYPE,ENTRY_DATE,POSTING_DATE,CLEARING_DOCUMENT,CLEARING_DATE,GL_ACCOUNT,AMOUNT_IN_DOC_CURR,DOCUMENT_CURRENCY,AMOUNT_IN_LOC_CURR_2,LOCAL_CURRENCY_2,AMOUNT_IN_LOCAL_CURRENCY,LOCAL_CURRENCY,TEXT,USER_NAME,COST_CENTER,CROSS_CO_CODE_NO,DOCUMENT_DATE,DOCUMENT_HEADER_TEXT,FISCAL_YEAR,MPM_FAMILY_NUMBER,MPM_ISSUE_NUMBER,MPM_PRODUCT_NUMBER,OFFSETTING_ACCT_NO,ORDER_1,PAYMENT_BLOCK,PAYMENT_DATE,PAYMENT_REFERENCE,POSTING_PERIOD,PROFIT_CENTER,PURCHASING_DOCUMENT,QUANTITY,REFERENCE,REFERENCE_KEY_1,REFERENCE_KEY_2,REFERENCE_KEY_3,REVERSED_WITH,VALUATED_AMT_LOC_CURR_2,WBS_ELEMENT,YEAR_MONTH,ADV_FILENAME,ADV_LOADED_BY,ADV_LOAD_DATE


In [26]:
appended_data = []

for one_file in paths:
    
    doc = codecs.open(one_file,'rU','UTF-8')
    print(one_file)
    
    final_df = pd.DataFrame()
    new_df = pd.DataFrame()
    
    df = pd.read_csv(one_file,skiprows=5,sep=';',header=None,skipinitialspace=True)
    df = pd.DataFrame(df[0].str.split('|', expand=True))
    df = df.loc[df[3].notnull()]
    df = df[df[3].str.contains('[0-9]',na=False)]

    if df.shape[1] == 44:
        #drop extra columns in import

        #Rename Columns
        new_df = df.drop([0,43],axis=1)
        new_df.columns = new_column_names

        #change datatype
        new_df = change_datatype(new_df)

        #Add filename as column
        new_df['ADV_FILENAME'] = os.path.basename(one_file)
        new_df['ADV_LOADED_BY'] = loaded_by
        new_df['ADV_LOAD_DATE'] = pd.Timestamp.now()
        
    else:
        
        #with column pc
        df_pc = df[~df[40].str.contains(',')]
        df_pc = df_pc.drop([0,38,39,45],axis=1)
        df_pc.columns = new_column_names
        df_pc = change_datatype(df_pc)


        #without column pc
        df_non_pc = df[df[40].str.contains(',')]
        df_non_pc = df_non_pc.drop([0,43,44,45],axis=1)
        df_non_pc.columns = new_column_names
        df_non_pc = change_datatype(df_non_pc)

        final_df = df_pc.append(df_non_pc,ignore_index=True)
        final_df.sort_values(by=['COMPANY_CODE', 'POSTING_DATE'],inplace=True)

        #Add filename as column
        final_df['ADV_FILENAME'] = os.path.basename(one_file)
        final_df['ADV_LOADED_BY'] = loaded_by
        final_df['ADV_LOAD_DATE'] = pd.Timestamp.now()
        
    super_final_df = new_df.append(final_df,ignore_index=True)
    appended_data.append(super_final_df)

#     del(df)
#     del(final_df)
#     del(new_df)
#     del(df_pc)
#     del(df_non_pc)
#     del(super_final_df)

appended_data = pd.concat(appended_data)

C:\Users\sdash029\Desktop\Project Folder\Project Cardinal\02242021\FY18 1496 OFFSET\2500_2018.csv
C:\Users\sdash029\Desktop\Project Folder\Project Cardinal\02242021\FY18 1496 OFFSET\2504_2018.csv
C:\Users\sdash029\Desktop\Project Folder\Project Cardinal\02242021\FY18 1496 OFFSET\6007_2018.csv
C:\Users\sdash029\Desktop\Project Folder\Project Cardinal\02242021\FY18 1496 OFFSET\6008_2018.csv
C:\Users\sdash029\Desktop\Project Folder\Project Cardinal\02242021\FY18 1496 OFFSET\6013_2018.csv
C:\Users\sdash029\Desktop\Project Folder\Project Cardinal\02242021\FY18 1496 OFFSET\6014_2018.csv
C:\Users\sdash029\Desktop\Project Folder\Project Cardinal\02242021\FY18 1496 OFFSET\6501_2018.csv
C:\Users\sdash029\Desktop\Project Folder\Project Cardinal\02242021\FY18 1496 OFFSET\6505_2018.csv
C:\Users\sdash029\Desktop\Project Folder\Project Cardinal\02242021\FY18 1496 OFFSET\6509_2018.csv
C:\Users\sdash029\Desktop\Project Folder\Project Cardinal\02242021\FY18 1496 OFFSET\8003_2018.csv
